# Wrappers example

We will see on this notebook what are the wrappers defined by sinergym and how to use them. Currently, we have developed a **normalization wrapper**,
**multi-observation wrapper** and **Logger wrapper**

In [ ]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)

In [ ]:
env_with_wrappers = MultiObsWrapper(LoggerWrapper(NormalizeObservation(gym.make('Eplus-demo-v1'))))

In [ ]:
env_with_wrappers = NormalizeObservation(gym.make('Eplus-demo-v1'))

In [ ]:
env_with_wrappers = LoggerWrapper(NormalizeObservation(gym.make('Eplus-demo-v1')))

.. warning:: The order of wrappers if you are going to use several at the same time is really important.
             The correct order is **Normalization - Logger - MultiObs** and subsets (for example, *Normalization* - *Multiobs* is valid).

.. note:: For more information about Sinergym Logger, visit :ref:`Logger`.

Now we just simply use the environment with the wrappers, for example:

In [ ]:
for i in range(1):
    obs = env_with_wrappers.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env_with_wrappers.action_space.sample()
        obs, reward, done, info = env_with_wrappers.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env_with_wrappers.close()